In [1]:
import torch
import sys
import os
import numpy as np
import PIL as Image
import matplotlib
matplotlib.use('qtagg')
from matplotlib import pyplot as plt
from torch.utils.data import random_split
sys.path.append(os.path.join(os.getcwd(), 'src'))

import model
import train_loop
import data_loader
import visual

In [2]:
DATA_DIR = os.path.join(os.getcwd(), "data")
TARGET_DATA_DIR = os.path.join(DATA_DIR, "target")
LIVECELL_IMG_DIR = os.path.join(DATA_DIR, "livecell", "images")
LIVECELL_MASK_DIR = os.path.join(DATA_DIR, "livecell", "masks")
UNITY_IMG_DIR = os.path.join(DATA_DIR, "unity_data", "images")
UNITY_MASK_DIR = os.path.join(DATA_DIR, "unity_data", "masks")

### liveCell dataset

Next cell creates dataset using only LiveCell dataset and divides dataset to train and test datasets. 

**Do not run this cell if you want to use combination of LivCell and synthetic dataset.**

In [ ]:
dataset = data_loader.MaskedDataset(LIVECELL_IMG_DIR, LIVECELL_MASK_DIR, length=None, in_memory=False)
seed = 123
test_percent = 0.001
n_test = int(len(dataset) * test_percent)
n_train = len(dataset) - n_test
train_set, test_set = random_split(dataset, [n_train, n_test], generator=torch.Generator().manual_seed(123))


### Combined dataset

Next cell creates dataset which combines LiveCell dataset with synthetic dataset and divides dataset to train and test datasets. 

**Do not run this cell if you ran previous code cell to use.**

In [ ]:
LC_dataset = data_loader.MaskedDataset(LIVECELL_IMG_DIR, LIVECELL_MASK_DIR, length=None, in_memory=False)
Unity_dataset = data_loader.MaskedDataset(UNITY_IMG_DIR, UNITY_MASK_DIR, length=None, in_memory=False)
datasets = [LC_dataset, Unity_dataset]
dataset = torch.utils.data.ConcatDataset(datasets)

seed = 123
test_percent = 0.001
n_test = int(len(dataset) * test_percent)
n_train = len(dataset) - n_test
train_set, test_set = random_split(dataset, [n_train, n_test], generator=torch.Generator().manual_seed(123))
    

# UNet 

### Unet training

In this part, you can create neural network and train it. This might take several hours, especially if you do not have GPU. You can change training parameters in `train.net()`.

If you have already trained model, there is no need to run this part.

In [ ]:
neural_net = model.Unet(numChannels=1, classes=2, dropout = 0.1)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
neural_net.to(device=device)
neural_net.train()

In [ ]:
train_loop.train_net(   net=neural_net,
                        dataset = train_set,
                        epochs= 1,              # Set epochs
                        batch_size= 2,          # Batch size
                        learning_rate=0.001,    # Learning rate
                        device=device,
                        val_percent=0.1,        # Percent of test set
                        save_checkpoint = True,
                        amp=False)

### Loading model

You can load already trained model from your computer. Write the path to model file in variable `PATH`.

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
PATH = ""

In [ ]:
net = torch.load(PATH, map_location=device)

### Visualisation

In [ ]:
visual.imshow_side_by_side_model(1, test_set, net)

In [ ]:
with torch.no_grad():
      image, mask = test_set[1]
      prediction = net(torch.unsqueeze(image, dim=0))
      prediction = torch.squeeze(prediction, dim=0)

In [ ]:
#plt.imshow(torch.squeeze(image), cmap = 'gray')
mask_np = np.delete(prediction.permute(1,2,0).numpy(),0, axis=2)

#mask_np = np.where(mask_np == 0, np.array([0,0,0]), mask_np)
#mask_np = np.where(mask_np == 1, np.array([0,1,0]), mask_np)
plt.imshow(mask_np, alpha = 1)

In [ ]:
plt.imshow(torch.squeeze(image), cmap = 'gray')

In [ ]:
plt.imshow(torch.squeeze(mask))

### Target domain visualisation


In [ ]:
target_set = data_loader.UnMaskedDataset(TARGET_DATA_DIR)

In [ ]:
with torch.no_grad():
      image = target_set[0]
      print(type(image))
      prediction = net(torch.unsqueeze(image, dim=0))
      prediction = torch.squeeze(prediction, dim=0)

In [ ]:
plt.imshow(torch.squeeze(image), cmap = 'gray')
mask_np = np.delete(prediction.permute(1,2,0).numpy(),0, axis=2)

mask_np = np.where(mask_np == 0, np.array([0,0,0]), mask_np)
mask_np = np.where(mask_np == 1, np.array([0,1,0]), mask_np)
plt.imshow(mask_np, alpha = 0.2)

In [ ]:
plt.imshow(torch.squeeze(image), cmap = 'gray')

# Domain Adaptation

## CycleGan

### Load trained model 

If you have allready trained model, you can use this section to load the model.

In [3]:
from CycleGan import Generator, ResBlock

In [4]:
G_A2B_Path = os.path.join(os.getcwd(), "model", "CycleGan_2022-03-19",  "test_resize_G_A2B.pt" )
G_B2A_Path = os.path.join(os.getcwd(), "model", "CycleGan_2022-03-19", "test_resize_G_B2A.pt" )

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
G_A2B = torch.load(G_A2B_Path, map_location=device)
G_B2A = torch.load(G_B2A_Path, map_location=device)

In [6]:
Target_dataset = data_loader.UnMaskedDataset(TARGET_DATA_DIR, mode=2)

seed = 123
target_test_percent = 0.01
n_test_target = int(len(Target_dataset) * target_test_percent)
n_train_target = len(Target_dataset) - n_test_target
target_train_set, target_test_set = torch.utils.data.random_split(Target_dataset, [n_train_target, n_test_target], generator=torch.Generator().manual_seed(seed))

In [7]:
with torch.no_grad():
        image = target_test_set[0]
        
        prediction = G_B2A(torch.unsqueeze(image, dim=0))
        prediction2 = G_A2B(prediction)
        prediction = torch.squeeze(prediction, dim=0)
        prediction2 = torch.squeeze(prediction2, dim=0)

In [8]:
prediction = prediction.permute(1,2,0)
prediction2 = prediction2.permute(1,2,0)

In [10]:
fig = plt.figure(figsize= 2 * np.array(plt.rcParams['figure.figsize']))
plt.subplot(1,3,1)
plt.imshow(torch.squeeze(image), cmap = 'gray')
plt.title("Original target image")
plt.subplot(1,3,2)
plt.imshow(prediction, cmap = 'gray')
plt.title("Shifted domain")
plt.subplot(1,3,3)
plt.imshow(prediction2, cmap = 'gray')
plt.title("domain shifted back to original")
plt.show()

KeyboardInterrupt: 

## Resizing image

In [ ]:
import matplotlib.image as mpimg
from torchvision import transforms
import transformer

In [ ]:
img = mpimg.imread(os.path.join(TARGET_DATA_DIR, "000_0.png"))

In [ ]:
image = transformer.to_same_size(image, img)

In [ ]:
fig = plt.figure(figsize= 2 * np.array(plt.rcParams['figure.figsize']))
plt.subplot(1,2,1)
plt.imshow(img)
plt.subplot(1,2,2)
plt.imshow(torch.squeeze(image), cmap = 'gray')
plt.show()